#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# Load the text file
loader = TextLoader("speech.txt")
documents = loader.load()


# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

In [9]:
embeddings=(OllamaEmbeddings(model="llama3.2:1b"))
# Create a vector store from the documents
db=FAISS.from_documents(docs, embeddings)

In [10]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content

'here and there and without countenance except from a lawless and malignant few.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [11]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'here and there and without countenance except from a lawless and malignant few.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [13]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='9c3dc5d3-6971-4fae-bcdc-5bfc9a642079', metadata={'source': 'speech.txt'}, page_content='here and there and without countenance except from a lawless and malignant few.'),
  np.float32(5289.337)),
 (Document(id='3b895473-71ec-4ce0-9092-c28f0460d427', metadata={'source': 'speech.txt'}, page_content='conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
  np.float32(5328.3574)),
 (Document(id='446f9c76-db69-498b-a8da-bc63b9e59735', metadata={'source': 'speech.txt'}, page_content='when those rights have been made as secure as the faith and the freedom of nations can make them.'),
  np.float32(5409.967)),
 (Document(id='f6dbcbff-1a81-4d6c-9d84-c545fe9caad1', metadata={'source': 'speech.txt'}, page_content='of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(5450.28

In [14]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-1.9788928031921387,
 2.699834108352661,
 0.5316368937492371,
 -0.9356720447540283,
 4.555887222290039,
 -4.531937599182129,
 -0.9959313869476318,
 2.5887506008148193,
 -0.02248937077820301,
 1.3163388967514038,
 -0.7299500703811646,
 -2.420287609100342,
 -0.6568418145179749,
 1.0130279064178467,
 2.9542837142944336,
 -0.9645138382911682,
 0.7148181200027466,
 0.9845128059387207,
 7.243058204650879,
 0.1253190040588379,
 -1.2049612998962402,
 1.7458763122558594,
 0.19492362439632416,
 -0.7355985641479492,
 -0.5432344675064087,
 3.289212942123413,
 -3.599170207977295,
 2.5917599201202393,
 -0.5567356944084167,
 1.98103928565979,
 -0.46658167243003845,
 1.953764796257019,
 -1.6541253328323364,
 2.0783984661102295,
 0.26889610290527344,
 -0.317547470331192,
 -1.0752415657043457,
 -2.187833547592163,
 -0.4503210186958313,
 -1.0603938102722168,
 -1.5395722389221191,
 -0.2545546293258667,
 -0.4901615083217621,
 2.261428117752075,
 -0.20802365243434906,
 0.5345672369003296,
 1.89838337898254

In [15]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='9c3dc5d3-6971-4fae-bcdc-5bfc9a642079', metadata={'source': 'speech.txt'}, page_content='here and there and without countenance except from a lawless and malignant few.'),
 Document(id='3b895473-71ec-4ce0-9092-c28f0460d427', metadata={'source': 'speech.txt'}, page_content='conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='446f9c76-db69-498b-a8da-bc63b9e59735', metadata={'source': 'speech.txt'}, page_content='when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(id='f6dbcbff-1a81-4d6c-9d84-c545fe9caad1', metadata={'source': 'speech.txt'}, page_content='of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.')]

In [16]:
### Saving And Loading
db.save_local("faiss_db")

In [17]:
new_db=FAISS.load_local("faiss_db",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [18]:
docs

[Document(id='9c3dc5d3-6971-4fae-bcdc-5bfc9a642079', metadata={'source': 'speech.txt'}, page_content='here and there and without countenance except from a lawless and malignant few.'),
 Document(id='3b895473-71ec-4ce0-9092-c28f0460d427', metadata={'source': 'speech.txt'}, page_content='conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='446f9c76-db69-498b-a8da-bc63b9e59735', metadata={'source': 'speech.txt'}, page_content='when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(id='f6dbcbff-1a81-4d6c-9d84-c545fe9caad1', metadata={'source': 'speech.txt'}, page_content='of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.')]